In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.salary import SalaryDataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, root_mean_squared_error

from mmpfn.models.tabpfn_v2 import TabPFNClassifier
from mmpfn.models.dino_v2.models.vision_transformer import vit_base
from mmpfn.models.tabpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.tabpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune.finetune_tabpfn_main import fine_tune_tabpfn

/home/wall/anaconda3/envs/mmpfn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/salary")
dataset = SalaryDataset(data_path)

In [3]:
accuracy_scores = []
for seed in range(10):
    torch.manual_seed(seed)
    # np.random.seed(seed)
    # print(f"Finetuning with seed: {seed}")
    
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]

    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_tabpfn_pad_ufes_20.ckpt"
    
    fine_tune_tabpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = TabPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
    )

    clf_finetuned = model_finetuned.fit(X_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 21:57:58,262] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   3%|▎         | 3/100 [00:01<01:06,  1.46it/s, Best Val. Loss=1.23, Best Val. Score=-1.23, Training Loss=1.23, Val. Loss=1.23, Patience=49, Utilization=0, Grad Norm=3.05][2025-09-17 21:57:59,482] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  40%|████      | 40/100 [00:24<00:38,  1.58it/s, Best Val. Loss=1.22, Best Val. Score=-1.22, Training Loss=1.18, Val. Loss=1.22, Patience=13, Utilization=0, Grad Norm=4.65][2025-09-17 21:58:23,111] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [01:04,  1.55it/s, Best Val. Loss=1.21, Best Val. Score=-1.21, Training Loss=1.2, Val. Loss=1.21, Patience=-47, Utilization=0, Grad Norm=4.65]                          
[2025-09-17 21:59:01,971] INFO - Initial Validation Loss: 1.2310269125488225 Best Validation L

accuracy_score (Finetuned): 0.45729166666666665


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 21:59:03,646] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:00<00:54,  1.80it/s, Best Val. Loss=1.21, Best Val. Score=-1.21, Training Loss=1.28, Val. Loss=1.21, Patience=50, Utilization=0, Grad Norm=nan][2025-09-17 21:59:04,172] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  33%|███▎      | 33/100 [00:20<00:44,  1.51it/s, Best Val. Loss=1.2, Best Val. Score=-1.2, Training Loss=1.2, Val. Loss=1.21, Patience=20, Utilization=0, Grad Norm=3.47]   [2025-09-17 21:59:23,991] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [01:03,  1.57it/s, Best Val. Loss=1.2, Best Val. Score=-1.2, Training Loss=1.15, Val. Loss=1.2, Patience=-47, Utilization=0, Grad Norm=7.8]                          
[2025-09-17 22:00:06,765] INFO - Initial Validation Loss: 1.2081275681665542 Best Validation Loss:

accuracy_score (Finetuned): 0.45


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 22:00:08,544] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:00<00:56,  1.73it/s, Best Val. Loss=1.25, Best Val. Score=-1.25, Training Loss=1.2, Val. Loss=1.25, Patience=50, Utilization=0, Grad Norm=nan][2025-09-17 22:00:09,052] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  37%|███▋      | 37/100 [00:22<00:41,  1.53it/s, Best Val. Loss=1.25, Best Val. Score=-1.25, Training Loss=1.23, Val. Loss=1.25, Patience=16, Utilization=0, Grad Norm=3.97][2025-09-17 22:00:31,113] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [01:03,  1.58it/s, Best Val. Loss=1.25, Best Val. Score=-1.25, Training Loss=1.12, Val. Loss=1.26, Patience=-47, Utilization=0, Grad Norm=5.02]                         
[2025-09-17 22:01:11,098] INFO - Initial Validation Loss: 1.2534227256683133 Best Validation Los

...

accuracy_score (Finetuned): 0.453125


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 22:04:26,185] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   3%|▎         | 3/100 [00:00<00:48,  2.01it/s, Best Val. Loss=1.28, Best Val. Score=-1.28, Training Loss=1.21, Val. Loss=1.28, Patience=49, Utilization=0, Grad Norm=2.76][2025-09-17 22:04:27,215] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  67%|██████▋   | 67/100 [00:41<00:19,  1.65it/s, Best Val. Loss=1.26, Best Val. Score=-1.26, Training Loss=1.23, Val. Loss=1.26, Patience=-14, Utilization=0, Grad Norm=4.21][2025-09-17 22:05:07,547] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [01:02,  1.60it/s, Best Val. Loss=1.26, Best Val. Score=-1.26, Training Loss=1.24, Val. Loss=1.27, Patience=-47, Utilization=0, Grad Norm=5.37]                         
[2025-09-17 22:05:28,135] INFO - Initial Validation Loss: 1.276273155536029 Best Validation L

accuracy_score (Finetuned): 0.46458333333333335


Fine-tuning Steps:   1%|          | 1/100 [00:00<?, ?it/s][2025-09-17 22:05:29,774] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   2%|▏         | 2/100 [00:00<00:56,  1.74it/s, Best Val. Loss=1.23, Best Val. Score=-1.23, Training Loss=1.3, Val. Loss=1.23, Patience=50, Utilization=0, Grad Norm=nan][2025-09-17 22:05:30,268] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  23%|██▎       | 23/100 [00:13<00:48,  1.60it/s, Best Val. Loss=1.22, Best Val. Score=-1.22, Training Loss=1.17, Val. Loss=1.22, Patience=30, Utilization=0, Grad Norm=3.28][2025-09-17 22:05:43,673] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  86%|████████▌ | 86/100 [00:53<00:08,  1.59it/s, Best Val. Loss=1.22, Best Val. Score=-1.22, Training Loss=1.2, Val. Loss=1.23, Patience=-32, Utilization=0, Grad Norm=5.31] 


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.4552083333333333
Std Accuracy: 0.009512875047359058


In [9]:
np.mean([0.45, 0.45729166666666665, 0.45729166666666665])

np.float64(0.4548611111111111)

In [10]:
np.std([0.45, 0.45729166666666665, 0.45729166666666665])

np.float64(0.003437324630767927)

In [11]:
np.mean([0.45729166666666665, 0.45, 0.4427083333333333, 0.4479166666666667, 0.45208333333333334])

np.float64(0.45)

In [12]:
np.mean([0.45729166666666665, 0.45, 0.4427083333333333, 0.4479166666666667, 0.45208333333333334])

np.float64(0.45)